#  Summarizer

summarizer preentrenado de Hugging Face (modelo google/pegasus-xsum)

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
from transformers import pipeline, AutoTokenizer
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

import os
from termcolor import colored
import tqdm
import scipy
import gc
import joblib

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab

pylab.rcParams['figure.figsize'] = 6, 4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Cargar los datos desde el archivo JSON
#data_path = '/content/drive/My Drive/proyecto/full_format_recipes.json'  # Clara
data_path = '/content/drive/My Drive/Colab Notebooks/proyecto/full_format_recipes.json'  # Jorge
recipes_df = pd.read_json(data_path)


Mounted at /content/drive


In [ ]:
import pandas as pd
from transformers import pipeline, AutoTokenizer

import pandas as pd
from transformers import pipeline, AutoTokenizer

# Cargar el tokenizer y el summarizer preentrenado
tokenizer = AutoTokenizer.from_pretrained('google/pegasus-xsum')
summarizer = pipeline("summarization", model='google/pegasus-xsum')

def summarize_directions(directions):
    # Verificar si directions es una lista no vacía o un string
    if isinstance(directions, list) and not directions:  # Si la lista está vacía
        return ""
    if pd.isna(directions).any():  # Usar .any() para comprobar si hay algún NaN
        return ""

    if isinstance(directions, list):
        # Si directions es una lista de instrucciones, resumir cada uno de sus elementos
        summaries = [summarizer(str(d), max_length=50, min_length=5, do_sample=False)[0]['summary_text'] for d in directions if d]
        return " ".join(summaries)  # Unir los resúmenes de los pasos
    elif isinstance(directions, str):
        # Si directions es un solo string, resumirlo directamente
        return summarizer(directions, max_length=50, min_length=5, do_sample=False)[0]['summary_text']
    else:
        return ""

# Seleccionar solo las primeras 5 recetas
subset_df = recipes_df.iloc[:5]

# Aplicar la función de resumen a la columna 'directions' de las primeras 5 recetas
subset_df['summarized_directions'] = subset_df['directions'].apply(summarize_directions)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Your max_length is set to 50, but your input_length is only 22. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Your max_length is set to 50, but your input_length is only 32. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
Your max_length is set to 50, but your input_length is only 30. Since this is a summarization task, where outputs shorter than the input are typically wanted, you migh

<ipython-input-8-f7290d0de6b8>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['summarized_directions'] = subset_df['directions'].apply(summarize_directions)


In [ ]:
# Función para mostrar el texto en bloques de palabras
def display_text_in_blocks(text, block_size=10):
    # Verificar si el texto es una lista de pasos (directions)
    if isinstance(text, list):
        for step in text:
            words = str(step).split()  # Convertir cada paso en string y luego dividirlo
            blocks = [" ".join(words[i:i+block_size]) for i in range(0, len(words), block_size)]
            for block in blocks:
                print(block)
            print()  # Separar cada paso por una línea en blanco
    else:
        words = text.split()
        blocks = [" ".join(words[i:i+block_size]) for i in range(0, len(words), block_size)]
        for block in blocks:
            print(block)

# Mostrar las direcciones y los resúmenes de forma organizada
for index, row in subset_df.iterrows():
    print(f"\nReceta {index + 1}:")
    print("\nDirecciones Originales:")
    display_text_in_blocks(row['directions'])  # Direcciones originales
    print("\nDirecciones Resumidas:")
    display_text_in_blocks(row['summarized_directions'])  # Direcciones resumidas
    print("\n" + "-"*80)  # Separador para mejorar la legibilidad


Receta 1:

Direcciones Originales:
1. Place the stock, lentils, celery, carrot, thyme, and salt
in a medium saucepan and bring to a boil. Reduce
heat to low and simmer until the lentils are tender,
about 30 minutes, depending on the lentils. (If they begin
to dry out, add water as needed.) Remove and discard
the thyme. Drain and transfer the mixture to a bowl;
let cool.

2. Fold in the tomato, apple, lemon juice, and olive
oil. Season with the pepper.

3. To assemble a wrap, place 1 lavash sheet on
a clean work surface. Spread some of the lentil mixture
on the end nearest you, leaving a 1-inch border. Top
with several slices of turkey, then some of the lettuce.
Roll up the lavash, slice crosswise, and serve. If using
tortillas, spread the lentils in the center, top with the
turkey and lettuce, and fold up the bottom, left side,
and right side before rolling away from you.


Direcciones Resumidas:
lentils, celery, carrot, thyme, and salt in a medium saucepan
and bring to a boil. In a l